In [1]:
import numpy as np
import pandas as pd
import h5py
import random

In [ ]:
def filter_non_informative(X, snp_ids, X_raw=None):
    tmp = X == X[0, :]
    a = (~tmp.all(0)).nonzero()[0]
    X = X[:, a]
    snp_ids = snp_ids[a]
    if X_raw is not None:
        X_raw = X_raw[:, a]
        return X, X_raw, snp_ids
    else:
        return X, snp_ids
    
def filter_duplicates(X, snp_ids, X_raw=None):
    uniques, index = np.unique(X, return_index=True, axis=1)
    X = uniques[:, np.argsort(index)]
    snp_ids = snp_ids[np.sort(index)]
    if X_raw is not None:
        X_raw = X_raw[:,np.sort(index)]
        return X, X_raw, snp_ids
    else:
        return X, snp_ids
    
def filter_maf(X, snp_ids, threshold, X_raw=None):
    freq = (np.sum(X, 0)) / (2 * X.shape[0])
    tmp = np.where(freq <= threshold)[0]
    X = np.delete(X, tmp, axis=1)
    snp_ids = np.delete(snp_ids, tmp, axis=0)
    if X_raw is not None:
        X_raw = np.delete(X_raw, tmp, axis=1)
        return X, X_raw, snp_ids
    else:
        return X, snp_ids

# load genotype matrix

In [ ]:
file_name="NAME_OF_GENOTYPE.h5"
with h5py.File(file_name, "r") as f:
    sample_ids = f['sample_ids'][:].astype(str)
    X = f['X_012'][:]
    snp_ids = f['snp_ids'][:].astype(str)

# compute simulations

In [ ]:
def get_simulation(X, sample_ids, snp_ids, number_of_samples, number_of_snps, explained_variance, maf, heritability, 
                   seed, number_background_snps, distribution, shape, effect):

    # sanity checks
    if effect=='add' and len(explained_variance) != number_of_snps:
        raise Exception('Need explained variance for each causative SNP!')
        
    # get random samples for X
    random.seed(seed)
    np.random.seed(seed)
    samples_to_take = random.sample(list(enumerate(sample_ids)), number_of_samples)
    sample_indices = np.array(samples_to_take)[:,0].astype(int)
    sample_ids_sampled = np.array(samples_to_take)[:,1]
    X_sampled = X[sample_indices,:]
    print('X after selection of samples ', X_sampled.shape)
    
    #filter non-informative
    X_sampled, snp_ids_sampled = filter_non_informative(X_sampled, snp_ids, X_raw=None)
    print('X after non-informative filter ', X_sampled.shape)
    
    # filter for duplicates
    X_sampled, snp_ids_sampled = filter_duplicates(X_sampled, snp_ids_sampled,  X_raw=None)
    print('X after duplicate filter ', X_sampled.shape)
    
    # filter for MAF
    X_sampled, snp_ids_sampled = filter_maf(X_sampled, snp_ids_sampled, maf, X_raw=None)
    print('X after maf filter ', X_sampled.shape)
    
    # compute simulations
    # choose random causal SNPs
    causal_snps = random.sample(list(enumerate(snp_ids_sampled)), number_of_snps)
    causal_snps_indices = np.array(causal_snps)[:,0].astype(int)
    causal_snps_ids = np.array(causal_snps)[:,1]

    # choose background SNPs
    X_non_causal = np.delete(X_sampled, causal_snps_indices, axis=1)
    snp_ids_non_causal = np.delete(snp_ids_sampled, causal_snps_indices, axis=0)
    background_SNPs_indices = np.random.choice(X_non_causal.shape[1], number_background_snps, replace=False)
    background_SNPs = X_non_causal[:,background_SNPs_indices]
    background_snp_ids = snp_ids_non_causal[background_SNPs_indices]

    # compute effect size for background
    betas_background = np.random.normal(loc=0, scale=0.1, size=number_background_snps)

    # add background
    simulated_phenotype = np.matmul(background_SNPs, betas_background)
    
    
    # set heritability
    background_variance = np.var(simulated_phenotype)
    noise_variance = background_variance/heritability-background_variance
    
    # add random noise 
    if distribution=='gamma':
        random_noise = np.random.gamma(shape=shape, scale=np.sqrt(noise_variance/shape), size=number_of_samples)
    elif distribution=='normal':
        random_noise = np.random.normal(loc=0, scale=np.sqrt(noise_variance), size=number_of_samples)
    simulated_phenotype = simulated_phenotype + random_noise

    #compute explained variances for more than 1 snp
    if effect == 'add' and number_of_snps>1:
        c = explained_variance[0]
        mean = (c*100) / number_of_snps
        sd = mean / c *10
        explained_variance = np.random.normal(mean,sd,number_of_snps)
        explained_variance = explained_variance/100
        explained_variance.sort()
        
    # add causative markers with effect sizes
    caus_beta = []
    for i in range(number_of_snps):
        beta = np.sqrt((explained_variance[i]/(1-explained_variance[i])*
                        (np.var(simulated_phenotype)/np.var(X_sampled[:,causal_snps_indices[i]]))))
        simulated_phenotype += beta*X_sampled[:,causal_snps_indices[i]]
        caus_beta.append(beta)
    if effect == 'mult':
        mult_snp = np.multiply(X_sampled[:,causal_snps_indices[0]], X_sampled[:,causal_snps_indices[1]])
        beta = np.sqrt((explained_variance[-1]/(1-explained_variance[-1])*
                                (np.var(simulated_phenotype)/np.var(mult_snp))))
        simulated_phenotype += beta*mult_snp
        caus_beta.append(beta)
                
    return simulated_phenotype, sample_ids_sampled, causal_snps_ids, background_snp_ids, betas_background, caus_beta, explained_variance


# some notes:
- the function is missing some sanity checks! Make sure all your parameters make sense!
- for each additional simulation add the corresponding parameters to the lists below. Then just run the cell below and the csv file with all the simulations will be generated
- number_of_simulation: the NAME of the simulation
- number_of_samples: amount of samples the simulated phenotype will have
- number_of_snps: amount of causative SNPs
- explained_variance: variance of causative SNPs
- shape: only relevant if distribution='gamma', shape of distribution
- distribution: 'normal' or 'gamma' are possible
- effect: 'add' for additive effect, 'mult' for multiplicative, 
    if 'mult' it computes: beta(ev1)$\cdot$SNP1 + beta(ev2)$\cdot$SNP2 + beta(ev3)$\cdot$SNP1$\cdot$SNP2
- will save the simulated phenotype, the simulated phenotype shifted to remove negative values, snp_ids of background SNPs and corresponding effect sizes(betas), a config file with the causal markers

# set parameters for simulations

In [ ]:
# parameters 
maf=0
heritability=[0.85, 0.7, 0.95]*3
number_background_snps=1000

number_of_simulation=[42, 43, 44, 45, 46, 47, 48, 49, 50]
number_of_samples=[1000]*9
number_of_snps=[20, 20, 20, 50, 50, 50, 100, 100, 100]
explained_variance=[[0.3]]*9
shape=[None]*9
distribution=['normal']*9
effect=['add']*9

# create and save all simulations specified in parameters

In [ ]:
causal_markers = []
seeds = []
background_markers = []
background_betas = []
causative_beta = []
ev = []

df_final = pd.DataFrame(index=sample_ids)

for i in range(len(number_of_simulation)):
    print('Simulation ', number_of_simulation[i])
    seed = 41 + number_of_simulation[i]
    simulated_phenotype, sample_ids_sampled, causal_snps_ids, background_snp_ids, betas_background, beta, c = get_simulation(X, 
                    sample_ids, snp_ids, number_of_samples[i], number_of_snps[i], explained_variance[i], maf, heritability[i], 
                       seed, number_background_snps, distribution[i], shape[i], effect[i])
    
    causal_markers.append(causal_snps_ids)
    seeds.append(seed)
    background_markers.append(background_snp_ids)
    background_betas.append(betas_background)
    causative_beta.append(beta)
    ev.append(c)
    
    df_sim = pd.DataFrame({f'sim{number_of_simulation[i]}': simulated_phenotype,
                    f'sim{number_of_simulation[i]}_shift': simulated_phenotype + np.abs(np.min(simulated_phenotype)) + 1},
                     index=sample_ids_sampled)
    df_final=df_final.join(df_sim)
    
df_final.to_csv(f'Simulation_{number_of_simulation[0]}-{number_of_simulation[-1]}.csv')

df_causal = pd.DataFrame({'simulation':number_of_simulation,
                         'seed': seeds,
                         'heritability': heritability,
                         'samples': number_of_samples,
                         'SNPs': number_of_snps,
                         'explained_var': ev,
                         'causal_marker': causal_markers,
                         'causal_beta': causative_beta,
                         'effect': effect,
                         'distribution': distribution,
                         'shape': shape})

df_causal.to_csv(f'simulation_config_{number_of_simulation[0]}-{number_of_simulation[-1]}.csv', index=None)


col = []
for sim in number_of_simulation:
    col.append('sim' + str(sim))
col = tuple(col)

bg = np.array(background_markers).T
df_background = pd.DataFrame(bg, columns=col)
df_background.to_csv(f'background_{number_of_simulation[0]}-{number_of_simulation[-1]}.csv', index=None)

bb = np.array(background_betas).T
df_bb = pd.DataFrame(bb, columns=col)
df_bb.to_csv(f'betas_background_{number_of_simulation[0]}-{number_of_simulation[-1]}.csv', index=None)